In [5]:
from langchain_ollama import OllamaLLM
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph

In [2]:
from typing import TypedDict,Annotated
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langgraph.graph import add_messages

In [3]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model='thinkverse/towerinstruct')

In [3]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage],add_messages]

In [9]:
def ChatBot(state:State):
    #Take user query
    query = state["messages"]
    
    #send to llm

    response = llm.invoke(query)
    #store to the state

    return {"messages":[response]}


In [10]:
from langgraph.graph import StateGraph,START,END
workflow =StateGraph(State)

#Add the nodes
workflow.add_node("chatbot",ChatBot)
# Add the edges
workflow.add_edge(START,"chatbot")
workflow.add_edge("chatbot",END)
# Compile the graph
memory =InMemorySaver()
graph =workflow.compile(checkpointer =memory)

In [14]:
res=graph.invoke(
    {"messages": [{"role": "user", "content": "What is my name"}]},
    {"configurable": {"thread_id": "1"}},
)

In [18]:
res

{'messages': [HumanMessage(content='hi! i am Bob', additional_kwargs={}, response_metadata={}, id='095337a0-0d62-4d69-8bf1-592b4e14d762'),
  HumanMessage(content=' I can understand and respond to your commands. What is your question or request?', additional_kwargs={}, response_metadata={}, id='1b97b925-c0de-4db9-9e73-2bb6fb910e9a'),
  HumanMessage(content='hi! i am Bob', additional_kwargs={}, response_metadata={}, id='07d6a029-2d3c-4728-b6c0-69b60c8d6a65'),
  HumanMessage(content=" Sure, what's your question or request?", additional_kwargs={}, response_metadata={}, id='e3d9ffe1-fec1-401e-b522-0516d51a807f'),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}, id='bc58ec8c-4696-40c3-a78e-49bad2396d97'),
  HumanMessage(content=' Your name is Bob.', additional_kwargs={}, response_metadata={}, id='501a1ffc-5c8c-40bc-a79d-9ed4a68a28f7')]}